<a href="https://colab.research.google.com/github/yeha98555/data-course-sample/blob/main/S4/A4_content-based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content-based

## 基礎建設

In [4]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [5]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-02 16:42:28--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.1’

All_Beauty.csv.1    100%[===================>]  14.78M  20.1MB/s    in 0.7s    

2022-01-02 16:42:29 (20.1 MB/s) - ‘All_Beauty.csv.1’ saved [15499476/15499476]

--2022-01-02 16:42:29--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.1’

meta_All_Beauty.jso 100%[===================>]   9.85M  15.2MB/s    in 0.6s   

In [6]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [7]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [8]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [9]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

### 去除完全相同的資料

metadata

In [10]:
print('Original Num of metadata: {0}'.format(len(metadata)))
print('Num of duplicated metadata: {0}'.format(metadata.duplicated(subset=['asin', 'title']).sum()))

Original Num of metadata: 32892
Num of duplicated metadata: 404


In [11]:
print('Drop duplicated metadata...')
metadata.drop_duplicates(subset=['asin', 'title'], keep=False, inplace=True)
print('Current Num of metadata: {0}'.format(len(metadata)))

Drop duplicated metadata...
Current Num of metadata: 32084


In [12]:
metadata = metadata.reset_index(drop=True)

ratings

In [13]:
print('Original Num of ratings: {0}'.format(len(ratings)))
print('Num of duplicated ratings: {0}'.format(ratings.duplicated().sum()))

Original Num of ratings: 371345
Num of duplicated ratings: 9070


In [14]:
print('Drop duplicated ratings...')
ratings.drop_duplicates(keep=False, inplace=True)
print('Current Num of ratings: {0}'.format(len(ratings)))

Drop duplicated ratings...
Current Num of ratings: 353351


In [15]:
ratings = ratings.reset_index(drop=True)

## 資料切分

In [17]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 欄位選擇（新增、剔除）

### 加入評分

In [18]:
rating_mean = ratings_trainings.groupby('asin')['overall'].mean().round(2).reset_index(name='RATE')
rating_count = pd.DataFrame(ratings_trainings.asin.value_counts())
rating_count.reset_index(inplace=True)
rating_count.columns = ['asin', 'RATE(COUNT)']
# merge
rating_data = rating_mean.merge(rating_count, on='asin')
rating_data

,asin,RATE,RATE(COUNT)
0,0061073717,5.00,2
1,0143026860,4.12,17
2,014789302X,4.35,20
3,0571348351,5.00,3
4,0692508988,5.00,1
...,...,...,...
32575,B01HIWLLUK,5.00,1
32576,B01HJ1K3YK,1.00,1
32577,B01HJ84SGM,5.00,5
32578,B01HJASD20,1.00,1


In [19]:
metadata = pd.merge(rating_data, metadata, left_on="asin", right_on="asin", how='right')

加入季節性標註
( None：表示商品無季節性 )

In [20]:
def MonthToSeason(x):   
    global season
    if x == 3 or x == 4 or x == 5:
         season = "Spring"
    elif x == 6 or x == 7 or x == 8:
         season = "Summer"
    elif x == 9 or x == 10 or x == 11:
         season = "Autumn"
    elif x == 12 or x == 1 or x == 2:
         season = "Winter"
    else:
         season = np.nan 
    return season

tmp_data = ratings_trainings.drop('unixReviewTime', axis=1)
tmp_data['SEASON'] = tmp_data['DATE'].dt.month.apply(lambda x : MonthToSeason(x))

In [21]:
tmp_data.head()

,asin,reviewerID,overall,DATE,SEASON
0,0143026860,A1V6B6TNIC10QE,1.0,2015-02-19,Winter
1,0143026860,A2F5GHSXFQ0W6J,4.0,2014-12-18,Winter
2,0143026860,A1572GUYS7DGSR,4.0,2014-08-10,Summer
3,0143026860,A1PSGLFK1NSVO,5.0,2013-03-11,Spring
4,0143026860,A6IKXKZMTKGSC,5.0,2011-12-25,Winter


In [22]:
tmp_data = tmp_data.groupby(['SEASON', 'asin']).size().reset_index(name='count')

tmp_data = tmp_data.pivot(index='asin',columns='SEASON',values='count').reset_index()
tmp_data.fillna(0.0,inplace=True)
tmp_data.set_index("asin" , inplace=True)

In [23]:
tmp_data['sum'] = tmp_data[['Autumn', 'Spring', 'Summer', 'Winter']].sum(axis=1).astype(int)

sum_threshold = 60
tmp_data = tmp_data[tmp_data['sum'] > sum_threshold]  # 希望至少有一定的銷售紀錄才做判別

tmp_data['mean'] = tmp_data[['Autumn', 'Spring', 'Summer', 'Winter']].mean(axis=1).round(2).astype(float)
tmp_data['std'] = tmp_data[['Autumn', 'Spring', 'Summer', 'Winter']].std(axis=1).round(3).astype(float)
tmp_data['fstd'] = tmp_data['std'] / tmp_data['mean']  # fixed std

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [24]:
tmp_data

SEASON,Autumn,Spring,Summer,Winter,sum,mean,std,fstd
asin,,,,,,,,
1620213982,1066.0,1070.0,1160.0,1496.0,4792,1198.00,203.352,0.169743
B000050B6B,47.0,30.0,43.0,34.0,154,38.50,7.853,0.203974
B000050B6H,22.0,15.0,22.0,33.0,92,23.00,7.439,0.323435
B000050FDT,23.0,24.0,15.0,24.0,86,21.50,4.359,0.202744
B000050FDY,374.0,396.0,421.0,347.0,1538,384.50,31.522,0.081982
...,...,...,...,...,...,...,...,...
B01H640HTG,37.0,42.0,34.0,29.0,142,35.50,5.447,0.153437
B01HBSH2EK,22.0,10.0,19.0,20.0,71,17.75,5.315,0.299437
B01HC6G4D6,24.0,52.0,22.0,40.0,138,34.50,14.177,0.410928


In [25]:
# 使用fstd篩選
season_tmp_data = tmp_data[tmp_data['fstd'] > 1]
print('以最低至少{0}的銷售紀錄為門檻時，得到的季節性商品占全部商品的約{1}%。\n'.format(sum_threshold, round(len(season_tmp_data) / len(tmp_data) * 100)))
print(season_tmp_data)

以最低至少60的銷售紀錄為門檻時，得到的季節性商品占全部商品的約2%。

SEASON      Autumn  Spring  Summer  Winter  sum   mean      std      fstd
asin                                                                     
B000I6AZHE     1.0   161.0     1.0     1.0  164  41.00   80.000  1.951220
B002GP80EU    93.0     4.0    10.0    13.0  120  30.00   42.166  1.405533
B002OXSKF8     2.0     0.0     1.0   104.0  107  26.75   51.506  1.925458
B00D8FUHFC    28.0     9.0   208.0    21.0  266  66.50   94.659  1.423444
B00KGKQR2Y     7.0    63.0    16.0    10.0   96  24.00   26.268  1.094500
B00RCNHRQ8    12.0    30.0    12.0    87.0  141  35.25   35.528  1.007887
B00U0PABWW     8.0    10.0    58.0     7.0   83  20.75   24.865  1.198313
B0189CM3U0     0.0    33.0     9.0   260.0  302  75.50  123.786  1.639550
B019FOXB70    10.0    84.0     7.0   291.0  392  98.00  133.504  1.362286
B01AUOTSVW     7.0    63.0    22.0     7.0   99  24.75   26.462  1.069172
B01B54XW54    24.0   260.0    72.0    15.0  371  92.75  114.273  1.232054
B

In [26]:
season_data = season_tmp_data.drop(['sum', 'mean', 'std', 'fstd'], axis=1)
season_data = season_data.idxmax(axis=1).reset_index(name='SEASON')
season_data

,asin,SEASON
0,B000I6AZHE,Spring
1,B002GP80EU,Autumn
2,B002OXSKF8,Winter
3,B00D8FUHFC,Summer
4,B00KGKQR2Y,Spring
5,B00RCNHRQ8,Winter
6,B00U0PABWW,Summer
7,B0189CM3U0,Winter
8,B019FOXB70,Winter
9,B01AUOTSVW,Spring


In [27]:
metadata = pd.merge(season_data, metadata, left_on="asin", right_on="asin", how='right')
metadata['SEASON'].fillna('None', inplace=True)

### 刪除非必要欄位

檢視各欄位分佈狀況（占有資料的占全部多少比例）。

In [28]:
import numpy as np
def product_columns_distribution(product_df):
    # replace empty with NaN
    product_df = product_df.replace('', np.nan)
    # replace empty list with NaN
    product_df.category = product_df.category.apply(lambda y: np.nan if len(y)==0 else y)
    product_df.description = product_df.description.apply(lambda y: np.nan if len(y)==0 else y)
    product_df.also_buy = product_df.also_buy.apply(lambda y: np.nan if len(y)==0 else y)
    product_df.feature = product_df.feature.apply(lambda y: np.nan if len(y)==0 else y)
    product_df.also_view = product_df.also_view.apply(lambda y: np.nan if len(y)==0 else y)
    product_df.imageURL = product_df.imageURL.apply(lambda y: np.nan if len(y)==0 else y)

    notnull_features = product_df.notnull()
    print(notnull_features.sum() / len(product_df['asin']) * 100)
    
product_columns_distribution(metadata)

asin               100.000000
SEASON             100.000000
RATE                99.975065
RATE(COUNT)         99.975065
category             0.000000
tech1                0.031168
description         45.334123
fit                  0.000000
title               99.996883
also_buy            20.312305
tech2                0.000000
brand               51.792171
feature              0.801022
rank               100.000000
also_view           24.535594
details            100.000000
main_cat           100.000000
similar_item         4.039397
date                 0.059220
price               35.023688
imageURL            50.308565
imageURLHighRes    100.000000
dtype: float64


In [29]:
metadata = metadata.drop(['category', 'tech1', 'fit', 'tech2', 'feature', 'date', 'imageURLHighRes'], axis=1)

展開欄位查看。

(1) `title`: 有一資料為空值，捨棄該筆資料。

In [30]:
metadata[metadata['title'] == '']

,asin,SEASON,RATE,RATE(COUNT),description,title,also_buy,brand,rank,also_view,details,main_cat,similar_item,price,imageURL
26209,B01A9MZLS4,None,4.5,16.0,[],,"[B00QGQTVHK, B00WRYQWW2, B00WRYP1WO, B01M122DM...",BCW,[],"[B00WRYQWW2, B00QGQTVHK, B00VUAYN70, B00W2N65F...",{'ASIN: ': 'B07DKSV8TC'},All Beauty,,$2.89 - $13.99,[https://images-na.ssl-images-amazon.com/image...


In [31]:
metadata = metadata.drop(metadata[metadata['title'] == ''].index).reset_index(drop=True)

(2) `main_cat`: 全部資料皆相同值，不具意義，捨去該欄位。

In [32]:
metadata['main_cat'].value_counts()

All Beauty    32083
Name: main_cat, dtype: int64

In [33]:
metadata = metadata.drop(['main_cat'], axis=1)

(3) `details`: 資料雜亂不齊全，選擇捨去該欄位。

In [34]:
metadata['details'].apply(pd.Series)

,ASIN:,Shipping Weight:,\n Item Weight: \n,Item model number:,UPC:,\n Product Dimensions: \n,Discontinued by manufacturer:,Domestic Shipping:,International Shipping:,Batteries,Shipping Advisory:,ASIN:
0,6546546450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7178680776,0.3 ounces (,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7250468162,3.5 ounces (,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7367905066,1.8 ounces,1.76 ounces,0000011837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7414204790,12 ounces (,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
32078,B01HIWLLUK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32079,B01HJ1K3YK,NaN,NaN,NaN,671635851871,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32080,B01HJ84SGM,1 pounds (,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32081,B01HJASD20,4.8 ounces (,NaN,NaN,739340584351,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
metadata = metadata.drop(['details'], axis=1)

(4) `rank`: 內含有商品類別和其商品於該類別排名，將其拆成RANK、CATEGORY兩欄位。

In [36]:
metadata[['RANK', 'CATEGORY']] = metadata['rank'].str.split(' in ', 1, expand=True)
metadata['CATEGORY'] = metadata['CATEGORY'].str.replace(' \(', '')
metadata['CATEGORY'] = metadata['CATEGORY'].str.replace('\&amp;', '&')
metadata['CATEGORY'].value_counts()

Beauty & Personal Care       31650
Grocery & Gourmet Food          36
Health & Household              21
Toys & Games                     3
Sports & Outdoors                3
Clothing, Shoes & Jewelry        2
Baby                             2
Tools & Home Improvement         1
Automotive                       1
Home & Kitchen                   1
Name: CATEGORY, dtype: int64

In [37]:
metadata = metadata.drop(['rank'], axis=1)

(5) `similar_item`: 資料為html表格，在每個`<th class="comparison_image_title_cell">`底下`<span aria-hidden="true" class="a-size-base a-color-base a-text-bold">`的`text`含有相似商品的title，將其格式轉換如`also_buy`、`also_view`。

In [38]:
from bs4 import BeautifulSoup
metadata['asin'] = metadata['asin'].astype(str)

def convert_similar_table_to_asin_list(html):
#     if (html == ''): return []
    asin_list = []
    
    soup = BeautifulSoup(html)
    
    compare_list = soup.findAll('th', {'class': 'comparison_image_title_cell'})
    for item in compare_list:
        title = item.span.text.strip()
        if (len(metadata[metadata['title'] == title]) > 0):
            asin = metadata[metadata['title'] == title]['asin'].iloc[0]
#             if (not any(asin.str in s for s in asin_list)):  # 去除重複
            asin_list.append(asin)
    
    return asin_list

In [39]:
metadata['SIMILAR_ITEM'] = metadata['similar_item'].apply(lambda x: convert_similar_table_to_asin_list(x) if x != ''  else x)

查看確認。

In [ ]:
metadata[metadata['SIMILAR_ITEM'] != ''][['similar_item', 'SIMILAR_ITEM']]

,similar_item,SIMILAR_ITEM
1,"class=""a-bordered a-horizontal-stripes a-spa...",[]
122,"class=""a-bordered a-horizontal-stripes a-spa...","[B005DMBC0C, B017WTVQI6]"
344,"class=""a-bordered a-horizontal-stripes a-spa...",[B015OI58OU]
455,"class=""a-bordered a-horizontal-stripes a-spa...",[B000FQMTOE]
458,"class=""a-bordered a-horizontal-stripes a-spa...",[]
...,...,...
32031,"class=""a-bordered a-horizontal-stripes a-spa...","[B01EBIY7HS, B01E82O696, B01F9XGGXW]"
32039,"class=""a-bordered a-horizontal-stripes a-spa...",[]
32044,"class=""a-bordered a-horizontal-stripes a-spa...",[]
32051,"class=""a-bordered a-horizontal-stripes a-spa...",[]


In [40]:
metadata = metadata.drop(['similar_item'], axis=1)

在剩下的欄位中，資料較齊全的有：`SEASON`、`RATE`、`RATE(COUNT)`、`title`、`rank`，規劃在方案1中選擇以這些特徵做為推薦參考。<br>
在方案2中，多考慮資料沒這麼不齊全但認為很重要的欄位：`description`、`brand`，將他們與`title`串接成新欄位`ALL_TEXT`，替代`title`作為推薦參考。<br>
在方案3中，以方案1、2中比較好為基底，新增如果商品未找到特徵計算得的相似商品，則以該商品`also_buy`、`also_view`、`similar_item`列表去推薦。<br>
其他像是`price`等，資料量太少，尚未想到使用方式，故暫時選擇捨去，或許未來能在思考如何使用。

In [41]:
metadata['SIMILAR'] = [set(list(x)+y+z) for x, y, z in zip(metadata.SIMILAR_ITEM, metadata.also_view, metadata.also_buy)]

In [42]:
metadata = metadata.drop(['SIMILAR_ITEM', 'also_view', 'also_buy'], axis=1)

In [43]:
metadata = metadata.drop(['price', 'imageURL'], axis=1)

In [ ]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32083 entries, 0 to 32082
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   asin         32083 non-null  object 
 1   SEASON       32083 non-null  object 
 2   RATE         32075 non-null  float64
 3   RATE(COUNT)  32075 non-null  float64
 4   description  32083 non-null  object 
 5   title        32083 non-null  object 
 6   brand        32083 non-null  object 
 7   RANK         31720 non-null  object 
 8   CATEGORY     31720 non-null  object 
 9   SIMILAR      32083 non-null  object 
dtypes: float64(2), object(8)
memory usage: 2.4+ MB


## 資料處理

### 文字處理

In [44]:
# 處理掉description 的中括號
metadata['description'] = metadata['description'].apply(lambda x: ' '.join(x))

# 合併三個文字欄位
metadata['ALL_TEXT'] = metadata['description'] + ' ' + metadata['brand'] + ' ' + metadata['title']

In [45]:
metadata = metadata.drop(['description', 'brand'], axis=1)

In [46]:
# 去除特殊符號，除了數字之間的
import re
regex = r"(?<!\d)[\W](?!\d)"
metadata['title'] = metadata['title'].apply(lambda x: re.sub(regex, ' ', str(x)))
metadata['ALL_TEXT'] = metadata['ALL_TEXT'].apply(lambda x: re.sub(regex, ' ', str(x)))

# 轉小寫
metadata['title'] = metadata['title'].str.lower()
metadata['ALL_TEXT'] = metadata['ALL_TEXT'].str.lower()

In [47]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# set lemmatizer
lemmatizer = WordNetLemmatizer()

# remove stopword
def remove_stopword(sentence):
    tokens = nltk.word_tokenize(sentence)
    result = [lemmatizer.lemmatize(voca) for voca in tokens if not voca in stopwords.words('english')]
    return ' '.join(result)

metadata['title'] = metadata['title'].apply(lambda x: remove_stopword(x))
metadata['ALL_TEXT'] = metadata['ALL_TEXT'].apply(lambda x: remove_stopword(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### TFIDF

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

#### 方案1: 只用title算tfidf-matrix¶

In [49]:
metadata1 = metadata.copy()

In [50]:
tmp_text = metadata1.drop_duplicates('title')
tf = TfidfVectorizer(max_features=200)
tfidf_matrix = tf.fit(tmp_text['title'])
tmp_word_features = tf.transform(metadata1['title'])

In [ ]:
# 查看tfidf_matrix資料，為調整TfidfVectorizer()參數。b
print(tmp_word_features.toarray())
print('=')
print(tf.get_feature_names())
print('=')
print(tf.vocabulary_)

In [51]:
word_features = pd.DataFrame(tmp_word_features.toarray(), columns = np.array(tf.get_feature_names()))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [52]:
metadata1 = pd.concat([metadata1, word_features], axis=1)

#### 方案2：用title、brand、description算tfidf-matrix

In [53]:
metadata2 = metadata.copy()

In [54]:
tmp_text = metadata2.drop_duplicates('ALL_TEXT')
tf = TfidfVectorizer(max_features=200)
tfidf_matrix = tf.fit(tmp_text['ALL_TEXT'])
tmp_word_features = tf.transform(metadata2['ALL_TEXT'])

In [55]:
word_features = pd.DataFrame(tmp_word_features.toarray(), columns = np.array(tf.get_feature_names()))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [56]:
metadata2 = pd.concat([metadata2, word_features], axis=1)

## 相似度計算

In [57]:
from sklearn.metrics.pairwise import cosine_similarity

#### 方案1

In [58]:
metadata1_features = metadata1.drop(['asin', 'SEASON', 'title', 'RANK', 'RATE', 'RATE(COUNT)', 'CATEGORY', 'SIMILAR', 'ALL_TEXT'], axis=1)

In [59]:
metadata1_similarity_matrix = cosine_similarity(metadata1_features)

#### 方案2

In [ ]:
metadata2_features = metadata2.drop(['asin', 'SEASON', 'title', 'RANK', 'RATE', 'RATE(COUNT)', 'CATEGORY', 'SIMILAR', 'ALL_TEXT'], axis=1)

In [ ]:
metadata2_similarity_matrix = cosine_similarity(metadata2_features)

## 若沒有相似商品的替代推薦

In [60]:
# 近一個月熱銷
recent_training_data = ratings_trainings[(ratings_trainings['DATE'] >= '2018-08-01') \
                                      & (ratings_trainings['DATE'] < '2018-09-01')]
def get_recent_topN(n=10):
    recent_top = recent_training_data.groupby('asin') \
           .agg({'asin':'size', 'overall':'mean'}) \
           .rename(columns={'asin':'sold_count','overall':'mean_score'}) \
           .reset_index().sort_values('sold_count', ascending=False) \
           .head(n).asin.reset_index(drop=True).to_list()
    return recent_top

In [ ]:
# TODO
# 長期熱銷
# asin_list找同CATEGORY的RANK排名前幾名
# 參考SIMILAR

## 產生推薦

In [61]:
mapping = pd.Series(metadata.index, index = metadata['asin'])

In [62]:
# 每個商品回傳 k 個最相近的商品
def recommend_item(similarity_matrix, item_input, k=10):
    try:
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (metadata['asin'].iloc[item_indices].tolist())
    except:
        return []

# 利用使用者購買過的商品產生推薦
def recommend_items(similarity_matrix, items, rmethod=1, k=10):
    res = []
    for d in items:
        res.extend(recommend_item(d, k))
        
    if len(res) != k:  # 如果相似商品們不足k個
        res = res + get_recent_topN(k-len(res))
    return res

In [63]:
def recommender(training_data, users=[], cmethod=1, rmethod=1, k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    ratings_trainings = training_data
    
    if (cmethod == 2):
        similarity_matrix = metadata2_similarity_matrix
    else:
        similarity_matrix = metadata1_similarity_matrix

#     recommendations = {user: recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist(), k) for user in users}
    recommendations = {user: recommend_items(similarity_matrix, metadata[metadata['asin'].isin(ratings_trainings[(ratings_trainings['reviewerID'] == user) & (ratings_trainings['overall'] > 3)]['asin'].tolist())]['title'].tolist(), rmethod, k) for user in users}

    return recommendations

## 結果評估

In [64]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

#### 方案1

In [65]:
for k in [10, 20, 30]:
    ratings_by_user = recommender(ratings_trainings, users, cmethod=1, rmethod=1, k=k)
    print(f'top{k} Recall： {evaluate(ratings_testings_by_user, ratings_by_user)}')

top10 Recall： 0.15593220338983052
top20 Recall： 0.19322033898305085
top30 Recall： 0.2627118644067797


In [66]:
metadata1 = metadata1[0:0]
metadata1_similarity_matrix = metadata1_similarity_matrix[0:0]
metadata1_features = metadata1_features[0:0]

#### 方案2

In [ ]:
for k in [10, 20, 30]:
    ratings_by_user = recommender(ratings_trainings, users, cmethod=2, rmethod=1, k=k)
    print(f'top{k} Recall： {evaluate(ratings_testings_by_user, ratings_by_user)}')